In [2]:
from time import sleep
from functools import cache
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np


def create_embeddings(texts):
    documents = [TaggedDocument(text, [i]) for i, text in enumerate(texts)]
    model = Doc2Vec(documents, vector_size=300, window=2, min_count=1, workers=4)
    embeddings = model.infer_vector([texts])

    return np.array(embeddings)

In [3]:
import pandas as pd

df = pd.read_csv('NAICS descriptions.csv')
embeds = []

for sector, definition, decscription in zip(df['Sector'], df['Definition'], df['Descriptions']):
    embeds.append({
        'label': sector,
        'definition': definition,
        'embed': create_embeddings(f'{definition} {decscription}')
    })

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

@cache
def create_embeddings(texts):
    documents = [TaggedDocument(text, [i]) for i, text in enumerate(texts)]
    model = Doc2Vec(documents, vector_size=300, window=2, min_count=1, workers=4)
    embeddings = model.infer_vector([texts])

    return list(embeddings)

def get_similarity(text):
    input_embedding = np.array(create_embeddings(str(text)))
    
    similarities = []
    for item in embeds:
        item_embedding = np.array(item['embed'])
        similarity_score = cosine_similarity(input_embedding.reshape(1, -1), item_embedding.reshape(1, -1))[0][0]
        similarities.append((similarity_score, item['label']))

    similarities.sort(key=lambda x: x[0], reverse=True)

    return [score for score, label in similarities]

# Example usage
similarities = get_similarity("some text")
similarities

[0.120325595,
 0.10622858,
 0.09881483,
 0.05922062,
 0.056757104,
 0.049552068,
 0.03743272,
 0.03685475,
 0.03407122,
 0.0332545,
 -0.0204064,
 -0.02041746,
 -0.024423823,
 -0.047419567,
 -0.048128556,
 -0.04916206,
 -0.059802324,
 -0.06468133,
 -0.06752068,
 -0.07298096]

In [5]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv('combined_df.csv')
new_rows = []

# Process all rows
for i in range(len(df)):
    readme = str(df['readme'][i])
    description = str(df['description'][i])
    try:    
        topic = ' '.join(literal_eval(str(df['topics'][i])))
    except:
        topic = str(df['topics'][i])
    if not topic:
        topic = 'not sure'
    
    label = df['NAICS Code'][i]
    
    new_row = {
        'readme': readme,
        'description': description,
        'label': label,
        'similarity_readme': get_similarity(readme),
        'similarity_description': get_similarity(description), 
        'similarity_topic': get_similarity(topic), 
        'embedding_readme': create_embeddings(readme),
        'embedding_description': create_embeddings(description),
        'embedding_topic': create_embeddings(topic),
    }
    new_rows.append(new_row)
    
    if i % 10 == 0:
        print(f'Finished {i} rows')

# Create a new DataFrame from the processed rows
combined_df = pd.DataFrame(new_rows)
combined_df.to_csv('combined_df_similarity.csv', index=False)

# Load and display the updated DataFrame
updated_df = pd.read_csv('combined_df_similarity.csv')
updated_df


Finished 0 rows
Finished 10 rows
Finished 20 rows
Finished 30 rows
Finished 40 rows
Finished 50 rows
Finished 60 rows
Finished 70 rows
Finished 80 rows
Finished 90 rows
Finished 100 rows
Finished 110 rows
Finished 120 rows
Finished 130 rows
Finished 140 rows
Finished 150 rows
Finished 160 rows
Finished 170 rows
Finished 180 rows
Finished 190 rows
Finished 200 rows
Finished 210 rows
Finished 220 rows
Finished 230 rows
Finished 240 rows
Finished 250 rows
Finished 260 rows
Finished 270 rows
Finished 280 rows
Finished 290 rows
Finished 300 rows
Finished 310 rows
Finished 320 rows
Finished 330 rows
Finished 340 rows
Finished 350 rows
Finished 360 rows
Finished 370 rows
Finished 380 rows
Finished 390 rows
Finished 400 rows
Finished 410 rows
Finished 420 rows
Finished 430 rows
Finished 440 rows
Finished 450 rows
Finished 460 rows
Finished 470 rows
Finished 480 rows
Finished 490 rows
Finished 500 rows
Finished 510 rows
Finished 520 rows
Finished 530 rows
Finished 540 rows
Finished 550 rows
Fin

,readme,description,label,similarity_readme,similarity_description,similarity_topic,embedding_readme,embedding_description,embedding_topic
0,"<div align=""center"">\n <a href=""https://erp...",Free and Open Source Enterprise Resource Plann...,['44-45'],"[0.11142118, 0.092933655, 0.08851982, 0.060225...","[0.15177895, 0.11709205, 0.10459731, 0.0808927...","[0.10136605, 0.084484145, 0.07513824, 0.064124...","[0.00035274684, 0.0016419973, -0.001092737, -0...","[-0.0008761545, 0.0008829641, -0.000964454, -0...","[0.0011661488, 0.0016442825, -0.0012225688, -0..."
1,# Shopizer 3.X (for java 1.8 +) (tested with J...,Shopizer java e-commerce software,['44-45'],"[0.07693073, 0.07016284, 0.0663882, 0.05846664...","[0.10583022, 0.08982481, 0.06565732, 0.0611231...","[0.11051515, 0.06632808, 0.053561576, 0.042051...","[0.0012771322, -0.0013735541, 0.00010561168, -...","[-0.0011958728, -0.0013463148, 0.0012915862, 0...","[0.0012713932, 0.00029528557, -0.00011824419, ..."
2,# Forecasting Best Practices \n\nTime series f...,Time Series Forecasting Best Practices & Examples,['44-45'],"[0.1423842, 0.12542576, 0.10062426, 0.04837413...","[0.092616186, 0.089335434, 0.07161768, 0.07037...","[0.08551658, 0.07180669, 0.062369697, 0.033399...","[0.001018378, 0.00035378098, -0.00013718089, 0...","[-0.00025009166, 0.0006035171, -0.0013919886, ...","[-0.0011748093, -0.0011219555, -1.3859769e-05,..."
3,## Retail Store on Blockchain\n\n### About\n\n...,:moneybag: Retail Store that runs on Ethereum,['44-45'],"[0.14935338, 0.116522655, 0.089538254, 0.06673...","[0.120838284, 0.09599927, 0.05700853, 0.051463...","[0.09268792, 0.08957516, 0.049509242, 0.042026...","[-0.0015838337, 0.0007894808, 0.0005901702, 0....","[0.0010532079, -0.00037138123, -0.0010101931, ...","[0.0003239874, -2.4107794e-05, 0.00070200226, ..."
4,# OpenPapyrus\n\nwww.petroglif.ru\n\n# En\n\n#...,"Sophisticated ERP, CRM, Point-Of-Sale, etc. Op...","['44-45', '42']","[0.07952741, 0.07908009, 0.07501243, 0.0715344...","[0.07949914, 0.05411622, 0.048660364, 0.033718...","[0.16071436, 0.11335474, 0.09230838, 0.0727709...","[-0.00087946287, 0.0007430651, 0.00030171633, ...","[0.00065273367, 0.0014273432, 0.0012474976, 0....","[0.0007940789, -0.0015974393, 0.00031665087, 0..."
...,...,...,...,...,...,...,...,...,...
1226,# Twitter-Sentiment-Analysis--Canadian-Electio...,Sentiment Analysis is a branch of Natural Lang...,['81'],"[0.10399457, 0.09368587, 0.091649264, 0.057013...","[0.110696495, 0.09393659, 0.07124597, 0.067505...","[0.06884805, 0.056082793, 0.05377169, 0.049393...","[-0.0012680298, -0.0006025531, -0.0008404162, ...","[-0.00034780285, -0.001443256, 0.0013935012, -...","[0.0009431922, 0.0015537872, 0.0011716873, -0...."
1227,# Sandstorm Specification\n\nSandstorm is an o...,Specification for an Organizational Tool for P...,['81'],"[0.12148962, 0.09282582, 0.060997363, 0.058109...","[0.10586621, 0.07415878, 0.0668112, 0.05940677...","[0.06884805, 0.056082793, 0.05377169, 0.049393...","[-0.0007569717, 0.00025186996, -0.0012480394, ...","[-0.0011845072, -0.0012923836, 0.0015792062, -...","[0.0009431922, 0.0015537872, 0.0011716873, -0...."
1228,"{'message': 'Not Found', 'documentation_url': ...",Old Wordpress Theme for a Football Political o...,['81'],"[0.12545882, 0.10552532, 0.10097495, 0.1009027...","[0.121807486, 0.11445299, 0.08072889, 0.062151...","[0.06884805, 0.056082793, 0.05377169, 0.049393...","[0.00034314394, -0.0008132763, -0.0010776301, ...","[-0.00096900074, 0.00036620814, 0.0006744506, ...","[0.0009431922, 0.0015537872, 0.0011716873, -0...."
1229,# Election Database Schema Design\nPerformance...,Performance of some analytics on real data fro...,['81'],"[0.12094249, 0.08727769, 0.080545254, 0.059966...","[0.10440619, 0.09485723, 0.07424985, 0.0576027...","[0.09940152, 0.05704648, 0.056884192, 0.043312...","[-0.000530715, -9.124557e-05, 0.0006625976, -0...","[-0.0015528599, 0.00060676836, 0.00019871732, ...","[0.0004962867, 0.0010608744, -0.00045351812, 0..."


In [6]:
import pandas as pd

df = df.dropna()
df = pd.read_csv('combined_df_similarity.csv')

finished_df = df.dropna()
finished_df.to_csv('finished_df.csv', index=False)

In [7]:
finished_df['similarity_description'][0]

'[0.15177895, 0.11709205, 0.10459731, 0.080892764, 0.06086638, 0.04756598, 0.039230075, 0.02630052, 0.016127238, 0.013462268, 0.008820651, 0.003802944, -0.011267061, -0.035623975, -0.036478, -0.045902472, -0.050302166, -0.09301818, -0.10696701, -0.1073946]'

In [8]:
import ast

convert = pd.read_csv('NAICS descriptions.csv').to_dict()['Sector']
convert = {v:k for k, v in convert.items()}

for index, row in finished_df.iterrows():
    vector = [0] * 20
    for j in ast.literal_eval(row['label']):
        if j == '0': continue 
        vector[convert[j]] = 1

    finished_df.at[index, 'label'] = vector

finished_df.to_csv('doc2vec_finished_df.csv', index=False)

In [9]:
finished_df.head()

,readme,description,label,similarity_readme,similarity_description,similarity_topic,embedding_readme,embedding_description,embedding_topic
0,"<div align=""center"">\n <a href=""https://erp...",Free and Open Source Enterprise Resource Plann...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.11142118, 0.092933655, 0.08851982, 0.060225...","[0.15177895, 0.11709205, 0.10459731, 0.0808927...","[0.10136605, 0.084484145, 0.07513824, 0.064124...","[0.00035274684, 0.0016419973, -0.001092737, -0...","[-0.0008761545, 0.0008829641, -0.000964454, -0...","[0.0011661488, 0.0016442825, -0.0012225688, -0..."
1,# Shopizer 3.X (for java 1.8 +) (tested with J...,Shopizer java e-commerce software,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.07693073, 0.07016284, 0.0663882, 0.05846664...","[0.10583022, 0.08982481, 0.06565732, 0.0611231...","[0.11051515, 0.06632808, 0.053561576, 0.042051...","[0.0012771322, -0.0013735541, 0.00010561168, -...","[-0.0011958728, -0.0013463148, 0.0012915862, 0...","[0.0012713932, 0.00029528557, -0.00011824419, ..."
2,# Forecasting Best Practices \n\nTime series f...,Time Series Forecasting Best Practices & Examples,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.1423842, 0.12542576, 0.10062426, 0.04837413...","[0.092616186, 0.089335434, 0.07161768, 0.07037...","[0.08551658, 0.07180669, 0.062369697, 0.033399...","[0.001018378, 0.00035378098, -0.00013718089, 0...","[-0.00025009166, 0.0006035171, -0.0013919886, ...","[-0.0011748093, -0.0011219555, -1.3859769e-05,..."
3,## Retail Store on Blockchain\n\n### About\n\n...,:moneybag: Retail Store that runs on Ethereum,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.14935338, 0.116522655, 0.089538254, 0.06673...","[0.120838284, 0.09599927, 0.05700853, 0.051463...","[0.09268792, 0.08957516, 0.049509242, 0.042026...","[-0.0015838337, 0.0007894808, 0.0005901702, 0....","[0.0010532079, -0.00037138123, -0.0010101931, ...","[0.0003239874, -2.4107794e-05, 0.00070200226, ..."
4,# OpenPapyrus\n\nwww.petroglif.ru\n\n# En\n\n#...,"Sophisticated ERP, CRM, Point-Of-Sale, etc. Op...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.07952741, 0.07908009, 0.07501243, 0.0715344...","[0.07949914, 0.05411622, 0.048660364, 0.033718...","[0.16071436, 0.11335474, 0.09230838, 0.0727709...","[-0.00087946287, 0.0007430651, 0.00030171633, ...","[0.00065273367, 0.0014273432, 0.0012474976, 0....","[0.0007940789, -0.0015974393, 0.00031665087, 0..."
